In [25]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [26]:
url_grimoire_root = 'https://www.destinypedia.com/Grimoire'

html = urlopen(url_grimoire_root)
bs = BeautifulSoup(html, 'html.parser')

In [27]:
list_gallary_text = bs.findAll('div',{'class': 'gallerytext'})

In [28]:
list_gallary_title = []
for gallary_text in list_gallary_text:
    gallary_text_re = re.sub(r'\n','', gallary_text.text)
    list_gallary_title.append(gallary_text_re)

In [29]:
list_gallary_title

['Guardian', 'Inventory', 'Allies', 'Enemies', 'Places', 'Activities']

In [32]:
list_section_df = []
list_title_df = []
list_subtitle_df = []
list_text_df = []

for i in range(len(list_gallary_title)):
    url_section = url_grimoire_root + ':' + list_gallary_title[i]
    section_html = urlopen(url_section)
    bs_section = BeautifulSoup(section_html, 'html.parser')
    
    list_title_text = bs_section.findAll('div',{'class': 'gallerytext'})
    
    list_title = []
    
    for title in list_title_text:
        title_text_re = re.sub(r'\n','', title.text)
        list_title.append(title_text_re)
        
    for j in range(len(list_title)):
        list_title[j] = re.sub("'",'%27',list_title[j])
        list_title[j] = re.sub("&",'%26',list_title[j])
        list_title[j] = re.sub(" ",'_',list_title[j])
        
        url_title = url_section + '/' + list_title[j]
        title_html = urlopen(url_title)
        bs_title = BeautifulSoup(title_html,'html.parser')
        
        select = bs_title.select_one('.mw-parser-output')

        flag = False
        
        text = ''

        for child in select:
            if child.name == 'h2':
            
                if flag == True:
                    list_text_df.append(text)
                    text = ''
            
                list_subtitle_df.append(child.text)
                list_title_df.append(list_title[j])
                list_section_df.append(list_gallary_title[i])
                flag = True
        
            elif flag == True and (child.name == 'p' or child.name == 'dl'):
                text += child.text
        
        list_text_df.append(text)
        
        list_title_df[j] = re.sub("%27","'",list_title_df[j])
        list_title_df[j] = re.sub("%26",'&',list_title_df[j])
        list_title_df[j] = re.sub("_",' ',list_title_df[j])
        
        

In [34]:
df_grimoire = pd.DataFrame({'section': list_section_df, 'title':list_title_df, 'sub_title':list_subtitle_df, 'text': list_text_df})

In [35]:
df_grimoire

,section,title,sub_title,text
0,Guardian,Classes,Guardians,"\n""Legends are carved into history by the brav..."
1,Guardian,Classes,Class: Hunter,"\n""Our old worlds have grown feral—rabid beast..."
2,Guardian,Classes,Ghost Fragment: Hunter,She leaves the Sparrow and climbs a long way a...
3,Guardian,Classes,Class: Titan,"\n""Stand. Not only to fight, but to strive. Fo..."
4,Guardian,Classes,Ghost Fragment: Titan,/ Tighten that strap.\n/ Eh?\n/ The gardbrace ...
...,...,...,...,...
834,Activities,Other_Activities,Festival of the Lost: Underwatch,"""Eri-i-i-is. Oh, Eri-i-i-is.""\n""Do you have my..."
835,Activities,Other_Activities,The Dawning,"“Pass me those lanterns, would you?” Eva said...."
836,Activities,Other_Activities,Dawning Fortunes,Fortune telling at the start of a new year is ...
837,Activities,Other_Activities,Haakon Precipice,"""Hey! Who left this forklift in the middle of ..."


In [ ]:
df_grimoire.to_json('./Database/raw_grimoire.json',orient='records')